In [ ]:
#hide
from transfertab.core import *
from nbdev.showdoc import *
from fastai.tabular.all import *

# transfertab

> Allow transfer learning using structured data.

## Install

`pip install transfertab`

## How to use

We have two fastai learners `learn1` and `learn2`  created, with models having embedding layers for their categorical variables.

In [ ]:
#hide
path = untar_data(URLs.ADULT_SAMPLE)
df1 = pd.read_csv(path/'adult.csv')
splits1 = RandomSplitter(valid_pct=0.2)(range_of(df1))
to1 = TabularPandas(df1, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits1)
dls1 = to1.dataloaders(bs=64)
learn1 = tabular_learner(dls1, metrics=accuracy)

#We add robot to our "race" column
new_rows = pd.DataFrame([[49,'Private',101320,'Assoc-acdm',12.0,'Married-civ-spouse','Exec-managerial','Wife','Robot','Female',0,1902,40,'United-States','>=50k'],
                        [18,'Private',182308,'Bachelors',10.0,'Never-married','?','Own-child','Other','Male',0,0,23,'United-States','<50k']],
                        columns=df1.columns)
df2 = df1.copy()
df2 = df2.append(new_rows, ignore_index=True)
df2.tail()

splits2 = RandomSplitter(valid_pct=0.2)(range_of(df2))
to2 = TabularPandas(df2, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits2)
dls2 = to2.dataloaders(bs=64)
learn2 = tabular_learner(dls2, metrics=accuracy)



In [ ]:
learn1, learn2, learn1.model

(<fastai.tabular.learner.TabularLearner at 0x7f85da8855b0>,
 TabularModel(
   (embeds): ModuleList(
     (0): Embedding(10, 6)
     (1): Embedding(17, 8)
     (2): Embedding(8, 5)
     (3): Embedding(16, 8)
     (4): Embedding(7, 5)
     (5): Embedding(6, 4)
     (6): Embedding(3, 3)
   )
   (emb_drop): Dropout(p=0.0, inplace=False)
   (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (layers): Sequential(
     (0): LinBnDrop(
       (0): Linear(in_features=42, out_features=200, bias=False)
       (1): ReLU(inplace=True)
       (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): LinBnDrop(
       (0): Linear(in_features=200, out_features=100, bias=False)
       (1): ReLU(inplace=True)
       (2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (2): LinBnDrop(
       (0): Linear(in_features=100, out_features=2, bias=True)
     )
   )
 ))

For transferring the embeddings from `learn1` to `learn2`, we first instantiate the `TabTransfer` class with the learners and the categorical variables to transfer.

In [ ]:
tabobj = TabTransfer(learn1, learn2)

Now, we can just call the transfer function to start the process.

In [ ]:
tabobj.transfer(["workclass"], verbose = True)

mean is tensor([ 0.0015,  0.0052,  0.0007, -0.0013,  0.0016,  0.0003],
       grad_fn=<MeanBackward1>) for Parameter containing:
tensor([[ 4.0982e-03,  1.0773e-02, -6.1128e-03, -1.2939e-02,  1.3693e-04,
          9.5658e-03],
        [-7.3064e-04, -9.0744e-03, -4.1995e-05,  3.8860e-03, -1.0992e-03,
         -9.0773e-03],
        [ 5.7799e-03,  1.1568e-03,  1.1390e-03,  6.0049e-03, -8.6575e-03,
          5.0584e-03],
        [ 6.2118e-03,  1.6866e-02,  1.3943e-02,  5.0229e-03,  1.2746e-02,
         -1.8922e-02],
        [-3.2620e-04,  6.3912e-03,  1.3365e-02,  1.4048e-02,  1.0076e-02,
          9.1949e-03],
        [-9.5517e-03,  7.0458e-03, -5.8635e-03, -5.2392e-03,  2.6704e-03,
          5.6363e-03],
        [ 4.7074e-03,  8.2458e-03, -1.2580e-02, -4.7994e-03,  3.7497e-03,
          7.2982e-04],
        [-7.3861e-03,  8.3785e-03, -4.2067e-03, -7.4433e-04, -3.3074e-03,
          5.5304e-03],
        [ 8.3096e-03,  7.7894e-03,  1.2156e-02, -9.9204e-03, -8.1630e-03,
         -1.5793e-02]